In [9]:
import folium
import pandas as pd
from branca.element import Template, MacroElement

pd.options.display.max_columns = 400
pd.options.display.max_rows = 400

rideshare = pd.read_csv('/Users/oliverbohler/Desktop/Springboard/RideShare/Code/rideshare_cleaned.csv')
rideshare.head()


,trip_id,year,month,day,day_name,start_date,end_date,start_time_new,end_time_new,duration,trip_route_category,passholder_type,bike_id,bike_type,start_station,start_station_name,start_station_region,start_lat,start_lon,end_station,end_station_name,end_station_region,end_lat,end_lon,num_of_docks
0,197704064,2022,7,1,Friday,2022-07-01,2022-07-01,00:00:00,00:21:00,21,One Way,Walk-up,19544,electric,4606,North Hollywood Station,North Hollywood,34.168629,-118.377068,4601,Lankershim & Hesby,North Hollywood,34.161709,-118.372818,20.0
1,197703662,2022,7,1,Friday,2022-07-01,2022-07-01,00:02:00,00:04:00,2,One Way,Monthly Pass,24302,electric,4406,Normandie & Hollywood,DTLA,34.101891,-118.300613,4442,Normandie & Sunset,DTLA,34.098000,-118.300468,14.0
2,197704062,2022,7,1,Friday,2022-07-01,2022-07-01,00:19:00,00:22:00,3,Round Trip,Walk-up,19941,electric,3082,Traction & Rose,DTLA,34.046520,-118.237411,3082,Traction & Rose,DTLA,34.046520,-118.237411,27.0
3,197704162,2022,7,1,Friday,2022-07-01,2022-07-01,00:23:00,00:25:00,2,Round Trip,Walk-up,17445,electric,3082,Traction & Rose,DTLA,34.046520,-118.237411,3082,Traction & Rose,DTLA,34.046520,-118.237411,27.0
4,197708062,2022,7,1,Friday,2022-07-01,2022-07-01,00:26:00,01:05:00,39,One Way,Walk-up,19941,electric,3082,Traction & Rose,DTLA,34.046520,-118.237411,3063,Pershing Square,DTLA,34.048038,-118.253738,27.0


In [10]:
rideshare['start_station_region'].value_counts()

start_station_region
DTLA               830116
Westside           544824
North Hollywood     14796
Name: count, dtype: int64

In [15]:
region_colors = {
    'DTLA': 'blue',
    'Westside': 'green',
    'North Hollywood': 'yellow'
}

rideshare['route'] = rideshare['start_station_name'] + " to " + rideshare['end_station_name']
route_frequencies = (
    rideshare.groupby(['start_station_name', 'route'])
    .size()
    .reset_index(name='route_count')
)
top_routes = (
    route_frequencies.sort_values('route_count', ascending=False)
    .groupby('start_station_name')
    .first()
    .reset_index()[['start_station_name', 'route']]
    .rename(columns={'start_station_name': 'station_name', 'route': 'top_routes'})
)
start_stations = rideshare[['start_station_name', 'start_station_region', 'start_lat', 'start_lon']].rename(
    columns={'start_station_name': 'station_name', 'start_station_region': 'region', 'start_lat': 'latitude', 'start_lon': 'longitude'}
)
end_stations = rideshare[['end_station_name', 'end_station_region', 'end_lat', 'end_lon']].rename(
    columns={'end_station_name': 'station_name', 'end_station_region': 'region', 'end_lat': 'latitude', 'end_lon': 'longitude'}
)
station_data = pd.concat([start_stations, end_stations]).drop_duplicates(subset=['station_name'])

avg_durations = (
    rideshare.groupby('start_station_name')['duration']
    .mean()
    .reset_index()
    .rename(columns={'start_station_name': 'station_name', 'duration': 'avg_trip_duration'})
)
station_data = station_data.merge(top_routes, on='station_name', how='left')
station_data = station_data.merge(avg_durations, on='station_name', how='left')
station_data['top_routes'] = station_data['top_routes'].fillna('No Frequent Route Available')
station_data['avg_trip_duration'] = station_data['avg_trip_duration'].fillna(0)

trip_volumes = (
    rideshare.groupby('start_station_name')['start_station_name']
    .count()
    .reset_index(name='trip_volume')
    .rename(columns={'start_station_name': 'station_name'})
)
station_data = station_data.merge(trip_volumes, on='station_name', how='left')
station_data['trip_volume'] = station_data['trip_volume'].fillna(0)
min_radius = 5
max_radius = 30
station_data['scaled_volume'] = (
    (station_data['trip_volume'] - station_data['trip_volume'].min()) /
    (station_data['trip_volume'].max() - station_data['trip_volume'].min())
) * (max_radius - min_radius) + min_radius

la_map = folium.Map(location=[34.0522, -118.2437], zoom_start=12)

legend_html = """
{% macro html(this, kwargs) %}
<div style="
    position: fixed; 
    bottom: 50px; left: 50px; 
    width: 200px; height: 120px; 
    background-color: white; 
    border:2px solid grey; 
    z-index:9999; 
    font-size:14px;
    padding: 10px;">
    <h4 style="margin-top: 0;">Region Legend</h4>
    <ul style="list-style-type: none; padding-left: 0;">
        <li><span style="background: blue; width: 12px; height: 12px; display: inline-block; border-radius: 50%;"></span> DTLA</li>
        <li><span style="background: green; width: 12px; height: 12px; display: inline-block; border-radius: 50%;"></span> Westside</li>
        <li><span style="background: yellow; width: 12px; height: 12px; display: inline-block; border-radius: 50%;"></span> North Hollywood</li>
    </ul>
</div>
{% endmacro %}
"""
macro = MacroElement()
macro._template = Template(legend_html)
la_map.get_root().add_child(macro)

#Circle Markers with Enhanced Popups
for _, row in station_data.iterrows():
    color = region_colors.get(row['region'], 'gray')

    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=row['scaled_volume'], 
        color=color, 
        fill=True, 
        fill_color=color, 
        fill_opacity=0.7,
        popup=(
            f"<b>Station:</b> {row['station_name']}<br>"
            f"<b>Region:</b> {row['region']}<br>"
            f"<b>Trips:</b> {row['trip_volume']}<br>"
            f"<b>Top Route:</b> {row['top_routes']}<br>"
            f"<b>Avg Trip Duration:</b> {round(row['avg_trip_duration'], 2)} mins"
        )
    ).add_to(la_map)

la_map.save('bike_station_map_with_top_routes.html')
la_map